In [4]:
from sklearn.model_selection import GridSearchCV
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

fer_directory = 'data/FER2013'
ck_directory = 'data/CK+'
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [ ]:
# Initialise generator with rescale factor 1./255
train_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)
test_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)

# Preprocess training set
train_generator = train_gen.flow_from_directory(
    fer_directory + '/train',
    target_size=(48,48),
    batch_size=64,
    color_mode='grayscale',
    class_mode='categorical'
)

# Preprocess test set
test_generator = test_gen.flow_from_directory(
    fer_directory + '/test',
    target_size=(48,48),
    batch_size=64,
    color_mode='grayscale',
    class_mode='categorical'
)

In [ ]:
# Grid search for optimal hyperparameters

# Define model
def create_model(learning_rate=0.001, dropout_rate=0.1, neurons=64):
    # Define model
    model = Sequential()

    # 1st convolution layer
    model.add(Conv2D(64, (3,3), padding='same', input_shape=(48,48,1)))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(dropout_rate))

    # 2nd convolution layer
    model.add(Conv2D(128, (5,5), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(dropout_rate))

    # 3rd convolution layer
    model.add(Conv2D(512, (3,3), padding='same'))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(dropout_rate))

    # Flatten and feed into dense layer
    model.add(Flatten())
    model.add(Dense(neurons))
    model.add(BatchNormalization())
    model.add(Activation('relu'))
    model.add(Dropout(dropout_rate))

    # Output layer
    model.add(Dense(7, activation='softmax'))

    # Compile model
    model.compile(optimizer=Adam(learning_rate=learning_rate, decay=1e-6), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Create model
experiment_model = KerasClassifier(model=create_model, epochs=10, verbose=2, dropout_rate=0.1, learning_rate=0.001, neurons=64)

# Define hyperparameters
dropout_rate = [0.1, 0.2, 0.3, 0.4]
neurons = [64, 128, 256]
learning_rate = [0.001, 0.01, 0.1]

# Split training set into X and y
(X_train, Y_train) = train_generator.next()
print(X_train.shape, Y_train.shape)
# Define grid search
param_grid = dict(dropout_rate=dropout_rate, neurons=neurons,learning_rate=learning_rate)
grid = GridSearchCV(estimator=experiment_model, param_grid=param_grid, n_jobs=1, cv=3, verbose=3)
grid_result = grid.fit(X_train, Y_train)

# Print results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, std, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, std, param))